In [1]:
### I ran this on a google colab, if running on a local machine, you might need to install different versions.

!pip install relbench[full]
!pip install pyg-lib -f https://data.pyg.org/whl/torch-2.4.0+cu121.html # PyG for working with graphs
!pip install git+https://github.com/pyg-team/pytorch_geometric.git # more PyG
!pip install pytorch_frame[full] #PyTorch Frame for working with tabular data

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.3/143.3 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.1/58.1 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 41.8 MB/s eta 0:00:00
Looking in links: https://data.pyg.org/whl/torch-2.4.0+cu121.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 20.1 MB/s eta 0:00:00
  Cloning https://github.com/pyg-team/pytorch_geometric.git to /tmp/pip-req-build-ndemswfa
  Running command git clone --filter=blob:none --quiet https://github.com/pyg-team/pytorch_geometric.git /tmp/pip-req-build-ndemswfa
  Resolved https://github.com/pyg-team/pytorch_geometric.git to commit 7f844d72d307ab9a105ba6494824cb0205ccbeac
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for torch-geometric: filename=torch_geometric-2.7.0-p

In [6]:
import os
import time

import torch
import numpy as np
import pandas as pd
import pooch
import pyarrow as pa
import pyarrow.json
from relbench.base import Database, Dataset, Table
from relbench.datasets import get_dataset, get_dataset_names, register_dataset
from relbench.utils import unzip_processor
from sklearn.impute import SimpleImputer

In [19]:
import duckdb
import pandas as pd

from relbench.base import Database, EntityTask, Table, TaskType
from relbench.datasets import get_dataset
from relbench.metrics import r2, mae
from relbench.tasks import get_task, get_task_names, register_task

In [8]:
BASE_DIR = '.'

In [5]:
customers = pd.read_csv(BASE_DIR + '/olist_customers_dataset.csv')
        #customers.drop(columns=['customer_id', 'customer_unique_id'], inplace=True)

orderItems = pd.read_csv('/kaggle/input/brazilian-ecommerce/olist_order_items_dataset.csv')
orderItems['order_item_id'] = orderItems.order_id.astype(str) + '-' + orderItems.order_item_id.astype(str)
orderItems["shipping_limit_date"] = pd.to_datetime(orderItems["shipping_limit_date"])

payments = pd.read_csv('/kaggle/input/brazilian-ecommerce/olist_order_payments_dataset.csv').reset_index().rename(columns={'index':'payment_id'})
payments['index'] = 'payment_id'

reviews = pd.read_csv('/kaggle/input/brazilian-ecommerce/olist_order_reviews_dataset.csv').drop_duplicates(subset=['review_id'])
reviews["review_creation_date"] = pd.to_datetime(reviews["review_creation_date"])
orders = pd.read_csv('/kaggle/input/brazilian-ecommerce/olist_orders_dataset.csv')
orders["order_approved_at"] = pd.to_datetime(orders["order_approved_at"])

products = pd.read_csv('/kaggle/input/brazilian-ecommerce/olist_products_dataset.csv')
sellers = pd.read_csv('/kaggle/input/brazilian-ecommerce/olist_sellers_dataset.csv')

FileNotFoundError: [Errno 2] No such file or directory: './olist_customers_dataset.csv'

In [ ]:
orderItems.isna().sum(axis=0)

In [9]:
### dataset available at https://www.kaggle.com/datasets/olistbr/brazilian-ecommerce

class EcommerceDataBase(Dataset):
    # example of creating your own dataset: https://github.com/snap-stanford/relbench/blob/main/tutorials/custom_dataset.ipynb

    val_timestamp = pd.Timestamp(year=2018, month=4, day=1)
    test_timestamp = pd.Timestamp(year=2018, month=8, day=1)

    def make_db(self) -> Database:

        tables = {}

        customers = pd.read_csv(BASE_DIR + '/olist_customers_dataset.csv').drop(columns=['customer_unique_id'])

        orderItems = pd.read_csv(BASE_DIR + '/olist_order_items_dataset.csv')
        orderItems['order_item_id'] = orderItems.order_id.astype(str) + '-' + orderItems.order_item_id.astype(str)
        orderItems["shipping_limit_date"] = pd.to_datetime(orderItems["shipping_limit_date"])

        payments = pd.read_csv(BASE_DIR + '/olist_order_payments_dataset.csv').reset_index().rename(columns={'index':'payment_id'})



        payments['index'] = 'payment_id'

        reviews = pd.read_csv(BASE_DIR + '/olist_order_reviews_dataset.csv').drop_duplicates(subset=['review_id']).fillna('')
        reviews["timestamp"] = pd.to_datetime(reviews["review_creation_date"])


        orders = pd.read_csv(BASE_DIR + '/olist_orders_dataset.csv').dropna()
        orders["timestamp"] = pd.to_datetime(orders["order_approved_at"])


        products = pd.read_csv(BASE_DIR + '/olist_products_dataset.csv')
        product_id = products['product_id']
        products['product_category_name'] = products['product_category_name'].fillna('')
        products_subset = products.drop(columns=['product_category_name', 'product_id'])
        products_imp = SimpleImputer().fit_transform(products_subset)
        products_imp = pd.DataFrame(products_imp, columns=products_subset.columns)
        products_imp['product_id'] = product_id
        products_imp['product_category_name'] = products['product_category_name']



        print(products_imp.isna().sum(axis=0))
        print(orders.isna().sum(axis=0))
        print(reviews.isna().sum(axis=0))
        print(payments.isna().sum(axis=0))
        print(orderItems.isna().sum(axis=0))
        print(customers.isna().sum(axis=0))



        sellers = pd.read_csv(BASE_DIR + '/olist_sellers_dataset.csv')

        tables['products'] = Table(
            df=pd.DataFrame(products_imp),
            pkey_col='product_id',
            fkey_col_to_pkey_table={},
            time_col=None
        )

        tables['customers'] = Table(
            df=pd.DataFrame(customers),
            pkey_col='customer_id',
            fkey_col_to_pkey_table={},
            time_col=None
        )

        tables['orderItems'] = Table(
            df=pd.DataFrame(orderItems),
            pkey_col='order_item_id',
            fkey_col_to_pkey_table={
                'order_id': 'orders',
                'product_id': 'products',
                'seller_id': 'sellers'
            },
            time_col=None
        )

        tables['payments'] = Table(
            df=pd.DataFrame(payments),
            pkey_col='payment_id',
            fkey_col_to_pkey_table={
                'order_id': 'orders'
            },
            time_col=None
        )

        tables['reviews'] = Table(
            df=pd.DataFrame(reviews),
            pkey_col='review_id',
            fkey_col_to_pkey_table={
                'order_id': 'orders'
            },
            time_col='timestamp'
        )

        tables['sellers'] = Table(
            df=pd.DataFrame(sellers),
            pkey_col='seller_id',
            fkey_col_to_pkey_table={}
        )

        tables['orders'] = Table(
            df=pd.DataFrame(orders),
            pkey_col='order_id',
            fkey_col_to_pkey_table={
                'customer_id': 'customers'
            },
            time_col='timestamp'
        )
        return Database(tables)

In [20]:
class SellerRevenueTask(EntityTask):
    # example of custom task: https://github.com/snap-stanford/relbench/blob/main/tutorials/custom_task.ipynb


    task_type = TaskType.REGRESSION
    entity_col = "seller_id"
    entity_table = "sellers"
    time_col = "timestamp"
    target_col = "revenue"
    timedelta = pd.Timedelta(days=30) # how far we want to predict revenue into the future.
    metrics = [r2, mae]
    num_eval_timestamps = 40

    def make_table(self, db: Database, timestamps: "pd.Series[pd.Timestamp]") -> Table:

        timestamp_df = pd.DataFrame({"timestamp": timestamps})

        orderItems = db.table_dict["orderItems"].df

        df = duckdb.sql(f"""
            select
                timestamp,
                seller_id,
                sum(price) as revenue
            from
                timestamp_df t
            left join
                orderItems oi
            on
                oi.shipping_limit_date <= t.timestamp + INTERVAL '{self.timedelta}'
                and oi.shipping_limit_date > t.timestamp
            group by timestamp, seller_id
        """).df().dropna()

        return Table(
            df=df,
            fkey_col_to_pkey_table={self.entity_col: self.entity_table},
            pkey_col=None,
            time_col=self.time_col,
        )

In [21]:
ecomm_ds = EcommerceDataBase()
db = ecomm_ds.get_db()

seller_revenue = SellerRevenueTask(ecomm_ds)

Making Database object from scratch...
(You can also use `get_dataset(..., download=True)` for datasets prepared by the RelBench team.)
product_name_lenght           0
product_description_lenght    0
product_photos_qty            0
product_weight_g              0
product_length_cm             0
product_height_cm             0
product_width_cm              0
product_id                    0
product_category_name         0
dtype: int64
order_id                         0
customer_id                      0
order_status                     0
order_purchase_timestamp         0
order_approved_at                0
order_delivered_carrier_date     0
order_delivered_customer_date    0
order_estimated_delivery_date    0
timestamp                        0
dtype: int64
review_id                  0
order_id                   0
review_score               0
review_comment_title       0
review_comment_message     0
review_creation_date       0
review_answer_timestamp    0
timestamp                  0
dty

### From here the code is pretty much taken from https://github.com/snap-stanford/relbench/blob/main/tutorials/train_model.ipynb

In [ ]:
train_table.df.isna().sum(axis=0)

timestamp    0
seller_id    0
revenue      0
dtype: int64

In [22]:
import numpy as np

from torch.nn import BCEWithLogitsLoss, L1Loss
from relbench.datasets import get_dataset
from relbench.tasks import get_task

train_table = seller_revenue.get_table("train")
val_table = seller_revenue.get_table("val")
test_table = seller_revenue.get_table("test")

out_channels = 1
loss_fn = L1Loss()
tune_metric = "mae"
higher_is_better = False

Making task table for train split from scratch...
(You can also use `get_task(..., download=True)` for tasks prepared by the RelBench team.)
Making Database object from scratch...
(You can also use `get_dataset(..., download=True)` for datasets prepared by the RelBench team.)
product_name_lenght           0
product_description_lenght    0
product_photos_qty            0
product_weight_g              0
product_length_cm             0
product_height_cm             0
product_width_cm              0
product_id                    0
product_category_name         0
dtype: int64
order_id                         0
customer_id                      0
order_status                     0
order_purchase_timestamp         0
order_approved_at                0
order_delivered_carrier_date     0
order_delivered_customer_date    0
order_estimated_delivery_date    0
timestamp                        0
dtype: int64
review_id                  0
order_id                   0
review_score               0
review_

In [23]:
import os
import math
import numpy as np
from tqdm import tqdm

import torch
import torch_geometric
import torch_frame

# Some book keeping
from torch_geometric.seed import seed_everything

seed_everything(42)


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)  # check that it's cuda if you want it to run in reasonable time!
root_dir = "./data"

cuda


In [24]:
from relbench.modeling.utils import get_stype_proposal

col_to_stype_dict = get_stype_proposal(db)

In [25]:
!pip install -U sentence-transformers # we need another package for text encoding
from typing import List, Optional
from sentence_transformers import SentenceTransformer
from torch import Tensor


class GloveTextEmbedding:
    def __init__(self, device: Optional[torch.device
                                       ] = None):
        self.model = SentenceTransformer(
            "sentence-transformers/average_word_embeddings_glove.6B.300d",
            device=device,
        )

    def __call__(self, sentences: List[str]) -> Tensor:
        return torch.from_numpy(self.model.encode(sentences))

In [26]:
from torch_frame.config.text_embedder import TextEmbedderConfig
from relbench.modeling.graph import make_pkey_fkey_graph

text_embedder_cfg = TextEmbedderConfig(
    text_embedder=GloveTextEmbedding(device=device), batch_size=256
)

data, col_stats_dict = make_pkey_fkey_graph(
    db,
    col_to_stype_dict=col_to_stype_dict,  # speficied column types
    text_embedder_cfg=text_embedder_cfg,  # our chosen text encoder
    cache_dir=os.path.join(
        root_dir, f"rel-ecomm_materialized_cache"
    ),  # store materialized graph for convenience
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/248 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/480M [00:00<?, ?B/s]

(…)beddings/whitespacetokenizer_config.json:   0%|          | 0.00/4.61M [00:00<?, ?B/s]

(…)WordEmbeddings/wordembedding_config.json:   0%|          | 0.00/164 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Embedding raw data in mini-batch: 100%|██████████| 13/13 [00:00<00:00, 154.47it/s]


In [27]:
from relbench.modeling.graph import get_node_train_table_input, make_pkey_fkey_graph
from torch_geometric.loader import NeighborLoader

loader_dict = {}

for split, table in [
    ("train", train_table),
    ("val", val_table),
    ("test", test_table),
]:
    table_input = get_node_train_table_input(
        table=table,
        task=seller_revenue,
    )
    entity_table = table_input.nodes[0]
    loader_dict[split] = NeighborLoader(
        data,
        num_neighbors=[
            128 for i in range(2)
        ],  # we sample subgraphs of depth 2, 128 neighbors per node.
        time_attr="time",
        input_nodes=table_input.nodes,
        input_time=table_input.time,
        transform=table_input.transform,
        batch_size=512,
        temporal_strategy="uniform",
        shuffle=split == "train",
        num_workers=0,
        persistent_workers=False,
    )

In [31]:
from torch.nn import BCEWithLogitsLoss
import copy
from typing import Any, Dict, List

import torch
from torch import Tensor
from torch.nn import Embedding, ModuleDict
from torch_frame.data.stats import StatType
from torch_geometric.data import HeteroData
from torch_geometric.nn import MLP
from torch_geometric.typing import NodeType

from relbench.modeling.nn import HeteroEncoder, HeteroGraphSAGE, HeteroTemporalEncoder


class Model(torch.nn.Module):

    def __init__(
        self,
        data: HeteroData,
        col_stats_dict: Dict[str, Dict[str, Dict[StatType, Any]]],
        num_layers: int,
        channels: int,
        out_channels: int,
        aggr: str,
        norm: str,
        # List of node types to add shallow embeddings to input
        shallow_list: List[NodeType] = [],
        # ID awareness
        id_awareness: bool = False,
    ):
        super().__init__()

        self.encoder = HeteroEncoder(
            channels=channels,
            node_to_col_names_dict={
                node_type: data[node_type].tf.col_names_dict
                for node_type in data.node_types
            },
            node_to_col_stats=col_stats_dict,
        )
        self.temporal_encoder = HeteroTemporalEncoder(
            node_types=[
                node_type for node_type in data.node_types if "time" in data[node_type]
            ],
            channels=channels,
        )
        self.gnn = HeteroGraphSAGE(
            node_types=data.node_types,
            edge_types=data.edge_types,
            channels=channels,
            aggr=aggr,
            num_layers=num_layers,
        )
        self.head = MLP(
            channels,
            out_channels=out_channels,
            norm=norm,
            num_layers=1,
        )
        self.embedding_dict = ModuleDict(
            {
                node: Embedding(data.num_nodes_dict[node], channels)
                for node in shallow_list
            }
        )

        self.id_awareness_emb = None
        if id_awareness:
            self.id_awareness_emb = torch.nn.Embedding(1, channels)
        self.reset_parameters()

    def reset_parameters(self):
        self.encoder.reset_parameters()
        self.temporal_encoder.reset_parameters()
        self.gnn.reset_parameters()
        self.head.reset_parameters()
        for embedding in self.embedding_dict.values():
            torch.nn.init.normal_(embedding.weight, std=0.1)
        if self.id_awareness_emb is not None:
            self.id_awareness_emb.reset_parameters()

    def forward(
        self,
        batch: HeteroData,
        entity_table: NodeType,
    ) -> Tensor:
        seed_time = batch[entity_table].seed_time
        x_dict = self.encoder(batch.tf_dict)

        rel_time_dict = self.temporal_encoder(
            seed_time, batch.time_dict, batch.batch_dict
        )

        for node_type, rel_time in rel_time_dict.items():
            x_dict[node_type] = x_dict[node_type] + rel_time

        for node_type, embedding in self.embedding_dict.items():
            x_dict[node_type] = x_dict[node_type] + embedding(batch[node_type].n_id)

        x_dict = self.gnn(
            x_dict,
            batch.edge_index_dict,
            batch.num_sampled_nodes_dict,
            batch.num_sampled_edges_dict,
        )

        return self.head(x_dict[entity_table][: seed_time.size(0)])

    def forward_dst_readout(
        self,
        batch: HeteroData,
        entity_table: NodeType,
        dst_table: NodeType,
    ) -> Tensor:
        if self.id_awareness_emb is None:
            raise RuntimeError(
                "id_awareness must be set True to use forward_dst_readout"
            )
        seed_time = batch[entity_table].seed_time
        x_dict = self.encoder(batch.tf_dict)
        # Add ID-awareness to the root node
        x_dict[entity_table][: seed_time.size(0)] += self.id_awareness_emb.weight

        rel_time_dict = self.temporal_encoder(
            seed_time, batch.time_dict, batch.batch_dict
        )

        for node_type, rel_time in rel_time_dict.items():
            x_dict[node_type] = x_dict[node_type] + rel_time

        for node_type, embedding in self.embedding_dict.items():
            x_dict[node_type] = x_dict[node_type] + embedding(batch[node_type].n_id)

        x_dict = self.gnn(
            x_dict,
            batch.edge_index_dict,
        )

        return self.head(x_dict[dst_table])


model = Model(
    data=data,
    col_stats_dict=col_stats_dict,
    num_layers=2,
    channels=128,
    out_channels=1,
    aggr="sum",
    norm="batch_norm",
).to(device)


# if you try out different RelBench tasks you will need to change these
optimizer = torch.optim.Adam(model.parameters(), lr=0.005)
epochs = 100

In [32]:
def train() -> float:
    model.train()

    loss_accum = count_accum = 0
    for batch in tqdm(loader_dict["train"]):
        batch = batch.to(device)

        optimizer.zero_grad()
        pred = model(
            batch,
            seller_revenue.entity_table,
        )
        pred = pred.view(-1) if pred.size(1) == 1 else pred

        loss = loss_fn(pred.float(), batch[entity_table].y.float())
        loss.backward()
        optimizer.step()

        loss_accum += loss.detach().item() * pred.size(0)
        count_accum += pred.size(0)

    return loss_accum / count_accum


@torch.no_grad()
def test(loader: NeighborLoader) -> np.ndarray:
    model.eval()

    pred_list = []
    for batch in loader:
        batch = batch.to(device)
        pred = model(
            batch,
            seller_revenue.entity_table,
        )
        pred = pred.view(-1) if pred.size(1) == 1 else pred
        pred_list.append(pred.detach().cpu())
    return torch.cat(pred_list, dim=0).numpy()

In [33]:
state_dict = None
best_val_metric = -math.inf if higher_is_better else math.inf
for epoch in range(1, epochs + 1):
    train_loss = train()
    val_pred = test(loader_dict["val"])
    val_metrics = seller_revenue.evaluate(val_pred, val_table)
    print(f"Epoch: {epoch:02d}, Train loss: {train_loss}, Val metrics: {val_metrics}")

    if (higher_is_better and val_metrics[tune_metric] > best_val_metric) or (
        not higher_is_better and val_metrics[tune_metric] < best_val_metric
    ):
        best_val_metric = val_metrics[tune_metric]
        state_dict = copy.deepcopy(model.state_dict())


model.load_state_dict(state_dict)
val_pred = test(loader_dict["val"])
val_metrics = seller_revenue.evaluate(val_pred, val_table)
print(f"Best Val metrics: {val_metrics}")

test_pred = test(loader_dict["test"])
test_metrics = seller_revenue.evaluate(test_pred)
print(f"Best test metrics: {test_metrics}")

100%|██████████| 21/21 [00:02<00:00,  8.53it/s]


Epoch: 01, Train loss: 835.8132757163645, Val metrics: {'r2': -0.18870422286850896, 'mae': 788.5052316122351}


100%|██████████| 21/21 [00:02<00:00,  8.81it/s]


Epoch: 02, Train loss: 825.9660947276402, Val metrics: {'r2': -0.1831251018051574, 'mae': 777.0489660069086}


100%|██████████| 21/21 [00:02<00:00,  7.20it/s]


...

Epoch: 98, Train loss: 386.93119152959713, Val metrics: {'r2': 0.3219976245426166, 'mae': 494.58683870899995}


100%|██████████| 21/21 [00:02<00:00,  7.97it/s]


Epoch: 99, Train loss: 389.8786452618914, Val metrics: {'r2': 0.3340590984073627, 'mae': 499.48525064165}


100%|██████████| 21/21 [00:02<00:00,  7.80it/s]


Epoch: 100, Train loss: 389.3405365555429, Val metrics: {'r2': 0.32796381346396175, 'mae': 503.7382699465967}
Best Val metrics: {'r2': 0.3267854005659607, 'mae': 494.7606130918582}
Best test metrics: {'r2': 0.28247078087920097, 'mae': 496.4684122518413}
